# Converting MAIAC HDF files to Geotiff 

Here we will go through a process to convert MAIAC hdf files to Geotiffs which can easily be read in and analyzed using R Studio. 

## Obtaining the MAIAC Data
MAIAC data is available from a number of sources online. We obtained the files using the [DAAC2Disk Web Tool](https://lpdaac.usgs.gov/tools/data_access/daac2disk_web), as it provided a way to intantly request and download multiple years' worth of MAIAC data instantly, rather than having to wait until a request is processed for up to a week as with Earthdata. Chicago is located in the MAIAC Sinusoidal tile (H04,V11). This script was executed on a folder containing all MAIAC files for that tile from 2016 and took around ten minutes to convert 2. 

## Conversion Function
This function converts the MAIAC hdf to a Geotiff file. In the filename line, it assumes standard MAIAC file naming conventions. It extracts only the Optical Depth 47 subdataset from a given HDF, but it can be easily modified to do the same with the quality assurance subdataset or create a multi-band Geotiff consisiting of all MAIAC layers. 

In [3]:
library(RCurl)
library(gdalUtils)
library(rgdal)
library(rts)
library(raster)
library(dplyr)
library(MODIStsp)

#Function to convert HDF to Geotiff (Run in Terminal or Jupyter Notebook, NOT RStudio)
HDF_to_GEOTIFF <- function (h) {
  hdf.file <- list.files(pattern = h, full.names = TRUE)
  
  sds <- get_subdatasets(hdf.file)
  
  #Choose the file you want to work with (Optical Depth 47)
  hdf.sub <- sds[1]
  hdf.sub
  #Creates a temporary outfile name
  tempname <- rasterTmpFile()  
  
  #Converts the hdf subdataset to a tif file. Returns NULL which is ok
  gdal_translate(hdf.sub, dst_dataset = tempname) 
  modis.r <- raster(tempname)
  
  wd <- getwd()
    
  filename = substr(hdf.sub,19,59)
  outname = paste(wd,filename, sep = "/") #Creates the path and file name for the output geotif. Change "wd" to path of output folder if different than working directory"
  writeRaster(modis.r, filename = outname, format = "GTiff", overwrite = TRUE)
}

Warning message:
“package ‘RCurl’ was built under R version 3.4.4”Loading required package: bitops
Warning message:
“package ‘gdalUtils’ was built under R version 3.4.4”Warning message:
“package ‘rgdal’ was built under R version 3.4.4”Loading required package: sp
rgdal: version: 1.3-4, (SVN revision 766)
 Geospatial Data Abstraction Library extensions to R successfully loaded
 Loaded GDAL runtime: GDAL 2.1.3, released 2017/20/01
 Path to GDAL shared files: /Users/isaackamber/Library/R/3.4/library/rgdal/gdal
 GDAL binary built with GEOS: FALSE 
 Loaded PROJ.4 runtime: Rel. 4.9.3, 15 August 2016, [PJ_VERSION: 493]
 Path to PROJ.4 shared files: /Users/isaackamber/Library/R/3.4/library/rgdal/proj
 Linking to sp version: 1.3-1 
Warning message:
“package ‘rts’ was built under R version 3.4.4”Loading required package: raster
Loading required package: xts
Warning message:
“package ‘xts’ was built under R version 3.4.4”Loading required package: zoo

Attaching package: ‘zoo’

The following objec

## Getting List of Files to Convert
The hdfs that will be converted are located in a folder "2016 Chicago MAIAC Data". That line can be modified based on the location of the files that are to be converted. Here, we see that there are 357 MAIAC hfds in that folder that we will soon convert.

In [16]:
#Get list of hdfs to convert
files <- list.files(path = "2016 Chicago MAIAC Data", pattern = "hdf")
#Get number of files in folder and view file names
summary(files)
head(files)

   Length     Class      Mode 
      357 character character 

[1] "MCD19A2.A2016002.h11v04.006.2018108081553.hdf"
[2] "MCD19A2.A2016003.h11v04.006.2018108081554.hdf"
[3] "MCD19A2.A2016004.h11v04.006.2018108081555.hdf"
[4] "MCD19A2.A2016005.h11v04.006.2018108081556.hdf"
[5] "MCD19A2.A2016006.h11v04.006.2018108081557.hdf"
[6] "MCD19A2.A2016007.h11v04.006.2018108081559.hdf"

## Convert the Files

In [17]:
for (i in 1:length(files)) {
  HDF_to_GEOTIFF(files[i])
}